In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow import keras
from keras.datasets import mnist

In [3]:
print(tf.__version__)

2.3.0


In [4]:
(train_data, train_label), (test_data, test_label) = mnist.load_data()

In [5]:
train_data = train_data / np.float32(255)
test_data = test_data / np.float32(255)

In [6]:
train_label = train_label.astype(np.int32)
test_label = test_label.astype(np.int32)

In [7]:
feature_columns = [tf.feature_column.numeric_column('x', shape = (28, 28))]

In [8]:
classifier = tf.estimator.LinearClassifier(feature_columns= feature_columns, n_classes=10, model_dir="../mnist_model/")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../mnist_model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x={"x":train_data}, y = train_label, batch_size = 100, num_epochs=None, shuffle=True)

In [10]:
classifier.train(input_fn=train_input_fn, steps=10)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../mnist_model/model.ckpt-10
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpo

In [11]:
val_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x = {'x' : test_data}, y = test_label, num_epochs=1, shuffle=False)

In [12]:
eval_results = classifier.evaluate(input_fn=val_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-08T16:32:29Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../mnist_model/model.ckpt-20
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.16737s
INFO:tensorflow:Finished evaluation at 2020-08-08-16:32:29
INFO:tensorflow:Saving dict for global step 20: accuracy = 0.8376, average_loss = 0.6038859, global_step = 20, loss = 0.6026582
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20: ../mnist_model/model.ckpt-20
{'accuracy': 0.8376, 'average_loss': 0.6038859, 'loss': 0.6026582, 'global_step': 20}


In [13]:
NB_CLASSES = 10
EPOCHS = 5
BATCH_SIZE = 128
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIMIZER = keras.optimizers.Adam()
IMG_ROWS, IMG_COLS = 28, 28
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)

In [14]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [15]:
X_train = X_train.reshape((-1, 28, 28, 1))
X_test = X_test.reshape((-1, 28, 28, 1))

In [16]:
X_train, X_test = X_train / 255.0, X_test / 255.0

In [17]:
X_train, X_test = X_train.astype('float32'), X_test.astype('float32')

In [18]:
y_train = keras.utils.to_categorical(y_train, NB_CLASSES)
y_test = keras.utils.to_categorical(y_test, NB_CLASSES)

In [19]:
def build(input_shape, classes):
    model = keras.models.Sequential([
        keras.layers.Convolution2D(20, (5, 5), activation='relu', input_shape = input_shape),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        keras.layers.Convolution2D(50, (5, 5), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(500, activation='relu'),
        keras.layers.Dense(classes, activation='softmax')
    ])
    
    return model

In [20]:
model = build(INPUT_SHAPE, NB_CLASSES)

In [21]:
model.compile(loss='categorical_crossentropy', optimizer = OPTIMIZER, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 20)        520       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 50)          25050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 500)               400500    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5

In [22]:
callbacks = [keras.callbacks.TensorBoard(log_dir='../logs')]

In [23]:
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size = BATCH_SIZE, verbose=VERBOSE, validation_split=VALIDATION_SPLIT, callbacks=callbacks)

Epoch 1/5
  1/375 [..............................] - ETA: 0s - loss: 2.3177 - accuracy: 0.0547WARNING:tensorflow:From /home/methodfunc/envs/tf2/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
375/375 [==============================] - 1s 3ms/step - loss: 0.2145 - accuracy: 0.9367 - val_loss: 0.0719 - val_accuracy: 0.9778
Epoch 2/5
375/375 [==============================] - 1s 3ms/step - loss: 0.0536 - accuracy: 0.9838 - val_loss: 0.0488 - val_accuracy: 0.9842
Epoch 3/5
375/375 [==============================] - 1s 3ms/step - loss: 0.0355 - accuracy: 0.9889 - val_loss: 0.0388 - val_accuracy: 0.9887
Epoch 4/5
375/375 [==============================] - 1s 4ms/step - loss: 0.0274 - accuracy: 0.9913 - val_loss: 0.0436 - val_accuracy: 0.9882
Epoch 5/5
375/375 [==============================] - 1s 4ms

In [24]:
score = model.evaluate(X_test, y_test, verbose=VERBOSE)

313/313 [==============================] - 0s 1ms/step - loss: 0.0290 - accuracy: 0.9906


In [25]:
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.028966721147298813
Test accuracy: 0.9905999898910522
